**feeder**

## Foto module (set maken met gezichten)

In [1]:
import cv2
import os
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display
from threading import Thread, Lock
import time
import numpy as np
import dlib
import random

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import shuffle

In [2]:
#RTSP_URL = "rtsp://wellantmilieu@gmail.com:q8oEZQC!W$8d@192.168.178.22:554/stream2"
cap = cv2.VideoCapture(1) #of 0

In [3]:
BASE_DIR = "datasets"
os.makedirs(BASE_DIR, exist_ok=True)

In [4]:
name_input = widgets.Text(description="Naam:")
set_name_button = widgets.Button(description="✅ Bevestig Naam")
image_widget = widgets.Image(format='jpeg')
capture_button = widgets.Button(description="📸 Foto maken", disabled=True)  # Eerst naam invoeren
stop_button = widgets.Button(description="🛑 Stop")
output = widgets.Output()

display(name_input, set_name_button, image_widget, capture_button, stop_button, output)

lock = Lock()

running = True
SAVE_DIR = None  # Wordt ingesteld na naam invoer

def update_stream():
    global running
    while running:
        with lock:
            ret, frame = cap.read()
        
        if ret:
            _, buffer = cv2.imencode('.jpg', frame)
            image_widget.value = buffer.tobytes()
        
        time.sleep(0.05)  # 20 FPS

def capture_photo(_):
    def save_image():
        if SAVE_DIR is None:
            with output:
                print("⚠️ Voer eerst een naam in en bevestig!")
            return
        
        with lock:
            ret, frame = cap.read()
        
        if ret:
            filename = f"{SAVE_DIR}/{name_input.value}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.jpg"
            cv2.imwrite(filename, frame)
            with output:
                print(f"✅ Foto opgeslagen: {filename}")
        else:
            with output:
                print("❌ Fout: Kon geen frame vastleggen!")

    Thread(target=save_image, daemon=True).start()

def set_name(_):
    global SAVE_DIR
    person_name = name_input.value.strip()

    if not person_name:
        with output:
            print("⚠️ Naam mag niet leeg zijn!")
        return
    
    SAVE_DIR = os.path.join(BASE_DIR, person_name)
    os.makedirs(SAVE_DIR, exist_ok=True)
    
    capture_button.disabled = False  # Activeer de foto maken knop
    with output:
        print(f"📂 Dataset-map ingesteld: {SAVE_DIR}")

def stop_stream(_):
    global running
    running = False
    with lock:
        cap.release()
    with output:
        print("🚨 Stream gestopt.")

set_name_button.on_click(set_name)
capture_button.on_click(capture_photo)
stop_button.on_click(stop_stream)

thread = Thread(target=update_stream, daemon=True)
thread.start()


Text(value='', description='Naam:')

Button(description='✅ Bevestig Naam', style=ButtonStyle())

Image(value=b'', format='jpeg')

Button(description='📸 Foto maken', disabled=True, style=ButtonStyle())

Button(description='🛑 Stop', style=ButtonStyle())

Output()

✅ Foto opgeslagen: datasets\Bobby/Bobby_20250314_125327.jpg
✅ Foto opgeslagen: datasets\Bobby/Bobby_20250314_125328.jpg
✅ Foto opgeslagen: datasets\Bobby/Bobby_20250314_125329.jpg
✅ Foto opgeslagen: datasets\Bobby/Bobby_20250314_125330.jpg
✅ Foto opgeslagen: datasets\Bobby/Bobby_20250314_125331.jpg
✅ Foto opgeslagen: datasets\Bobby/Bobby_20250314_125331.jpg
✅ Foto opgeslagen: datasets\Bobby/Bobby_20250314_125332.jpg
✅ Foto opgeslagen: datasets\Bobby/Bobby_20250314_125332.jpg
✅ Foto opgeslagen: datasets\Bobby/Bobby_20250314_125332.jpg
✅ Foto opgeslagen: datasets\Bobby/Bobby_20250314_125333.jpg
✅ Foto opgeslagen: datasets\Bobby/Bobby_20250314_125333.jpg
✅ Foto opgeslagen: datasets\Bobby/Bobby_20250314_125334.jpg
✅ Foto opgeslagen: datasets\Bobby/Bobby_20250314_125334.jpg
✅ Foto opgeslagen: datasets\Bobby/Bobby_20250314_125334.jpg
✅ Foto opgeslagen: datasets\Bobby/Bobby_20250314_125335.jpg
✅ Foto opgeslagen: datasets\Bobby/Bobby_20250314_125335.jpg
✅ Foto opgeslagen: datasets\Bobby/Bobby_

### FACE detection (dataset maken met alleen de gezichten)

In [5]:
# Ophalen van beschikbare dataset-mappen
available_datasets = [d for d in os.listdir(BASE_DIR) if os.path.isdir(os.path.join(BASE_DIR, d))]

# Dropdown-menu voor dataset-selectie
dataset_dropdown = widgets.Dropdown(
    options=available_datasets,
    description="Selecteer Naam:"
)
process_button = widgets.Button(description="Start Gezichtsdetectie")
output = widgets.Output()

# Weergave in Notebook
display(dataset_dropdown, process_button, output)

# Functie voor gezichtsdetectie
def process_faces(_):
    selected_name = dataset_dropdown.value
    if not selected_name:
        with output:
            print("Selecteer een dataset om te verwerken.")
        return

    DATASET_DIR = os.path.join(BASE_DIR, selected_name)
    PROCESSED_DIR = os.path.join(DATASET_DIR, "processed_faces")
    os.makedirs(PROCESSED_DIR, exist_ok=True)

    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    detector = dlib.get_frontal_face_detector()

    for filename in os.listdir(DATASET_DIR):
        img_path = os.path.join(DATASET_DIR, filename)
        img = cv2.imread(img_path)

        if img is None:
            with output:
                print(f"Kan afbeelding {filename} niet openen, overslaan.")
            continue

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = detector(gray)

        if len(faces) == 0:
            with output:
                print(f"Geen gezichten gevonden in {filename}, overslaan.")
            continue

        for i, face in enumerate(faces):
            x, y, w, h = face.left(), face.top(), face.width(), face.height()

            if w < 50 or h < 50:
                with output:
                    print(f"Klein gezicht gedetecteerd in {filename}, overslaan.")
                continue

            aspect_ratio = w / h
            if aspect_ratio < 0.8 or aspect_ratio > 1.2:
                with output:
                    print(f"Ongebruikelijk gezicht in {filename}, overslaan.")
                continue

            face_img = img[y:y+h, x:x+w]
            face_resized = cv2.resize(face_img, (100, 100))
            save_path = os.path.join(PROCESSED_DIR, f"{filename}_face{i}.jpg")
            cv2.imwrite(save_path, face_resized)

            with output:
                print(f"Gezicht uit {filename} opgeslagen als {save_path}")

    with output:
        print(f"Alle gezichten opgeslagen in {PROCESSED_DIR}")

# Koppel knop aan functie
process_button.on_click(process_faces)


Dropdown(description='Selecteer Naam:', options=('Arend', 'Bobby', 'PJ', 'Robert', 'Unknown'), value='Arend')

Button(description='Start Gezichtsdetectie', style=ButtonStyle())

Output()